# LDAP Dump Post-processing

The file `ldump.py` includes functions for creating the LDAP dump files and combining them into a single `attributes.pkl` file. From there, I used this notebook to help me visualize, filter, and export the data in CSV. The results are probably not reproducible yet.

In [1]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
%matplotlib inline

The shape of the data is one very long list of dicts, each dict representing key/value attributes that LDAP returned. Turning that directly into a dataframe gives us every single key as a column, and those who don't have a value for a column receive NaN.

In [2]:
with open('attributes.pkl', 'rb') as f:
    a = pickle.load(f)
df = pd.DataFrame(a)

In [ ]:
df

(above output not included for obvious reasons)

## LDAP Attributes

Here I wanted to look at what attributes are provided by LDAP, and how many of them area available for each user. This is by far the trickiest part of the LDAP import: determining which attributes are "universal" and which ones are missing for users that should be on the site. Additionally, how do we figure out whether an LDAP entry is for a student, faculty, or alum?  A summary of my findings is below (note that 278,000 entries are in the dump).

- The uid (case ID), sn (last name), givenName (first name), cn (full name) fields are present for pretty much every entry. The mail-related fields are also pretty much universal.
- The most useful field for determining the type of user seems to be `cwruEduPersonScopedAffiliation`. It only occurs for around 138,000 entries though. The other affiliation-related entries (first seen, last seen, expiration) are even less common. I have a suspicion that these fields only occur for users with the objectClass cwruEduPerson, which might make a good filter.
- The `cwruEduEntryLastModified` seems pretty universal as well, and I later use that to order and filter users by how recently they were modified. The assumption is that recently modified users are more relevant (i.e. have been to case within a few years).

In [ ]:
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')
print_full(df.count())

In [ ]:
df.dtypes

## Case ID Explorations

The LDAP attribute values are provided in lists, even though many attributes are single-valued. We'll be seing a lot of this pattern, where we apply a lambda that grabs the first item from a list.

In [ ]:
df.uid.apply(lambda x: x[0])

Here I was examining the distribution of Case IDs. It appears that most are between 3 and 6 digits of length, although 7 is possible (and legitimate). There are three outliers - two of length 8 and one of length 46. I manually removed these three.

In [ ]:
uidstr = df.uid.apply(lambda x: x[0])
uidstr.apply(len).argmax()

In [ ]:
uidstr[174025]

In [ ]:
idlens = uidstr.apply(len)
idlens.value_counts()

In [ ]:
idlens[idlens == 8]

In [ ]:
idlens[idlens == 46]

In [ ]:
uidstr[174025]

In [ ]:
df = df.drop([174025, 162555, 162770])

## Cleanup and filtering

Here begins the main segment of cleanup and filtering. In particular, I'm selecting a subset of the attributes I'm interested in (see below). Then, I'm making them all single valued. Finally, I'm filtering down to only users who have values for all of these attributes.

In [ ]:
cleaner = df[['uid', 'cn', 'sn', 'givenName', 'mail', 'cwruEduAccountAdded', 'cwruEduEntryLastModified', 'cwruEduPersonScopedAffiliation']]

In [ ]:
cleaner.uid = cleaner.uid.apply(lambda x: x[0])

In [ ]:
cleaner.loc[:, 'cn'] = cleaner.cn.apply(lambda x: x[0])

In [ ]:
cleaner.loc[:, 'sn'] = cleaner.sn.apply(lambda x: x[0])

I think one of the rows had a NaN for a name somewhere, and thus I dropped it. Grossss

In [ ]:
cleaner = cleaner.drop(269654)

In [ ]:
cleaner.loc[:, 'givenName'] = cleaner.givenName.apply(lambda x: x[0])

In [ ]:
cleaner.loc[:, 'cwruEduAccountAdded'] = cleaner.cwruEduAccountAdded.apply(lambda x: x[0][:8] if type(x) is list else x)

In [ ]:
cleaner.loc[:, 'cwruEduEntryLastModified'] = cleaner.cwruEduEntryLastModified.apply(lambda x: x[0][:8] if type(x) is list else x)

Here's the filtering...

In [ ]:
cleanest = cleaner.dropna()

In [ ]:
print(cleanest.shape)
cleanest = cleanest.sort_values('cwruEduAccountAdded')
cleanest

Now I'm manually checking some IDs to see how everything works.

In [ ]:
cleanest[cleanest.uid == 'hsc21']

In [ ]:
cleanest.shape

A blunt filtering mechanism: "if you were modified since last year, you probably belong in the database."  Not sure if this is the flawed assumption, or if a previous filtering step was problematic.

In [ ]:
recent = cleanest[cleanest.cwruEduEntryLastModified > '20160000']
recent.shape

In [ ]:
recent[recent.uid == 'exk92']

## Exporting

Here I take the existing dataframe and prepare it (rename, drop, create columns as necessary) to be in the format expected out of the import CSV.

In [ ]:
export = recent.rename(columns={'uid': 'username', 'sn': 'last_name', 'givenName': 'first_name'})
export = export.drop(['cn', 'mail', 'cwruEduAccountAdded', 'cwruEduEntryLastModified'], axis=1)
export

In [ ]:
def compute_department(affil):
    if 'student@case.edu' in affil:
        department = 'Student'
    elif 'faculty@case.edu' in affil:
        department = 'Faculty'
    elif 'alum@case.edu' in affil:
        department = 'Alumni'
    else:
        department = ''
    return department

export['department'] = export.cwruEduPersonScopedAffiliation.apply(compute_department)

In [ ]:
export = export.drop(['cwruEduPersonScopedAffiliation'], axis=1)

In [ ]:
export

In [ ]:
export['photo_url'] = 'https://placehold.it/100x100'

In [ ]:
export = export[['username', 'first_name', 'last_name', 'department', 'photo_url']]

In [ ]:
export.to_csv('employees.csv', index=False)

Due to performance limitations on the CSV import process, I split it into three CSVs, hoping that I could do them each sequentially.

In [ ]:
split = export.shape[0] // 3

In [ ]:
export1 = export.iloc[0:split]

In [ ]:
export2 = export.iloc[split:2 * split]

In [ ]:
export3 = export.iloc[2*split:]

In [ ]:
export1.to_csv('employees1.csv', index=False)

In [ ]:
export2.to_csv('employees2.csv', index=False)

In [ ]:
export3.to_csv('employees3.csv', index=False)